In [1]:
import duckdb as ddb
import pandas as pd

Establish a connection to the DuckDB database file and

con  # Display the connection object to verify it is active

In [2]:
con = ddb.connect("../air_quality.db")
con

In [3]:
df = con.query("select * from raw.air_quality where parameter in ('so2', 'pm10', 'pm25')").to_df()
#len(df)
df.head()

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
0,225405,1303524,Bongani Mabaso Eco Park-237701,2024-02-29 23:00:00,-26.77761,27.837328,pm25,µg/m³,22.095,03,2024,2026-01-25 00:05:00.764864
1,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 00:00:00,-26.77761,27.837328,pm25,µg/m³,23.414,03,2024,2026-01-25 00:05:00.764864
2,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 01:00:00,-26.77761,27.837328,pm25,µg/m³,25.818,03,2024,2026-01-25 00:05:00.764864
3,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 02:00:00,-26.77761,27.837328,pm25,µg/m³,30.793,03,2024,2026-01-25 00:05:00.764864
4,225405,1303524,Bongani Mabaso Eco Park-237701,2024-03-01 03:00:00,-26.77761,27.837328,pm25,µg/m³,36.194,03,2024,2026-01-25 00:05:00.764864


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   location_id         471 non-null    int64         
 1   sensors_id          471 non-null    int64         
 2   location            471 non-null    object        
 3   datetime            471 non-null    datetime64[us]
 4   lat                 471 non-null    float64       
 5   lon                 471 non-null    float64       
 6   parameter           471 non-null    object        
 7   units               471 non-null    object        
 8   value               471 non-null    float64       
 9   month               471 non-null    object        
 10  year                471 non-null    int64         
 11  ingestion_datetime  471 non-null    datetime64[us]
dtypes: datetime64[us](2), float64(3), int64(3), object(4)
memory usage: 44.3+ KB


In [5]:
df.describe()

,location_id,sensors_id,datetime,lat,lon,value,year,ingestion_datetime
count,471.0,4.710000e+02,471,471.00000,4.710000e+02,471.000000,471.0,471
mean,225405.0,1.303565e+06,2024-03-05 16:55:01.910828,-26.77761,2.783733e+01,16.887253,2024.0,2026-01-25 00:05:00.764863
min,225405.0,1.303524e+06,2024-02-29 23:00:00,-26.77761,2.783733e+01,0.000043,2024.0,2026-01-25 00:05:00.764864
25%,225405.0,1.303524e+06,2024-03-02 19:00:00,-26.77761,2.783733e+01,0.007850,2024.0,2026-01-25 00:05:00.764864
50%,225405.0,1.303541e+06,2024-03-05 13:00:00,-26.77761,2.783733e+01,13.184000,2024.0,2026-01-25 00:05:00.764864
75%,225405.0,1.303631e+06,2024-03-07 18:00:00,-26.77761,2.783733e+01,23.998000,2024.0,2026-01-25 00:05:00.764864
max,225405.0,1.303631e+06,2024-03-12 10:00:00,-26.77761,2.783733e+01,152.557000,2024.0,2026-01-25 00:05:00.764864
std,0.0,4.699906e+01,NaN,0.00000,7.112982e-15,19.694169,0.0,NaN


In [6]:
df.describe(include='O')

,location,parameter,units,month
count,471,471,471,471
unique,1,3,2,1
top,Bongani Mabaso Eco Park-237701,pm25,µg/m³,03
freq,471,157,314,471


In [7]:
df[df.duplicated(subset=["location_id","datetime", "parameter", "units", "value", "month", "year", "ingestion_datetime"])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime


In [8]:
df.groupby(by="parameter", as_index=False).count()

,parameter,location_id,sensors_id,location,datetime,lat,lon,units,value,month,year,ingestion_datetime
0,pm10,157,157,157,157,157,157,157,157,157,157,157
1,pm25,157,157,157,157,157,157,157,157,157,157,157
2,so2,157,157,157,157,157,157,157,157,157,157,157


In [9]:
df.groupby(by=["parameter", "units"], as_index=False).count()

,parameter,units,location_id,sensors_id,location,datetime,lat,lon,value,month,year,ingestion_datetime
0,pm10,µg/m³,157,157,157,157,157,157,157,157,157,157
1,pm25,µg/m³,157,157,157,157,157,157,157,157,157,157
2,so2,ppm,157,157,157,157,157,157,157,157,157,157


In [10]:
con.close()